In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import gym
import pathlib
import typing as tp
from scipy.special import softmax
import pandas as pd
import os

In [3]:
sys.path.insert(0, '/mnt/ubuntu_data_hdd/school/masters/uwaterloo/1b/cs885/project/codenames/codenames_ai/src')

In [ ]:
from default_game import *
from codenames import *
from codenames_env import *

[libprotobuf WARNING external/com_google_protobuf/src/google/protobuf/text_format.cc:339] Warning parsing text-format research_scann.ScannConfig: 43:11: text format contains deprecated field "min_cluster_size"
2022-11-18 17:10:08.708313: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 53044
2022-11-18 17:10:10.142973: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:88] PartitionerFactory ran in 1.434609309s.
[libprotobuf WARNING external/com_google_protobuf/src/google/protobuf/text_format.cc:339] Warning parsing text-format research_scann.ScannConfig: 43:11: text format contains deprecated field "min_cluster_size"
2022-11-18 17:10:23.111726: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 47240
2022-11-18 17:10:24.385631: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:88] PartitionerFactory ran in 1.273856681s.


In [5]:
import stable_baselines3
from stable_baselines3.common import env_checker
from stable_baselines3 import DQN, SAC, PPO, A2C, HerReplayBuffer
import matplotlib.pyplot as plt

from stable_baselines3 import TD3
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback, CallbackList

In [6]:
from stable_baselines3.common import env_checker

In [7]:
env_checker.check_env(CodenamesEnvHack(glove, wordlist))

/home/sherman/miniconda3/envs/fn_env/lib/python3.8/site-packages/stable_baselines3/common/env_checker.py:130: UserWarning: Your observation achieved_goal has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/home/sherman/miniconda3/envs/fn_env/lib/python3.8/site-packages/stable_baselines3/common/env_checker.py:130: UserWarning: Your observation desired_goal has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/home/sherman/miniconda3/envs/fn_env/lib/python3.8/site-packages/stable_baselines3/common/env_checker.py:130: UserWarning: Your observation observation has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector

In [8]:
from stable_baselines3.common.logger import configure
import pathlib

tmp_path = "/mnt/ubuntu_data_hdd/school/masters/uwaterloo/1b/cs885/project/codenames/sb3_log/"
# set up logger
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])
tmp_path_discrete = "/mnt/ubuntu_data_hdd/school/masters/uwaterloo/1b/cs885/project/codenames/sb3_log_discrete/"
# set up logger
new_logger_discrete = configure(tmp_path_discrete, ["stdout", "csv", "tensorboard"])

Logging to /mnt/ubuntu_data_hdd/school/masters/uwaterloo/1b/cs885/project/codenames/sb3_log/
Logging to /mnt/ubuntu_data_hdd/school/masters/uwaterloo/1b/cs885/project/codenames/sb3_log_discrete/


In [9]:
new_logger

In [10]:
def make_env(env_name):
    log_dir = pathlib.Path(tmp_path).joinpath(env_name)
    log_dir.mkdir(parents=True, exist_ok=True)
    return Monitor(CodenamesEnvHack(glove, wordlist), str(log_dir))

In [11]:
def make_env_discrete(env_name):
    log_dir = pathlib.Path(tmp_path_discrete).joinpath(env_name)
    return Monitor(CodenamesEnvHackDiscrete(glove, wordlist), str(log_dir))

In [12]:
def get_path_for(env_name, is_discrete=False):
    if is_discrete:
        return str(pathlib.Path(tmp_path_discrete).joinpath(env_name))
    return str(pathlib.Path(tmp_path).joinpath(env_name))

In [13]:
log_freq = 25000
num_train_steps = 1000000

In [15]:
n_sampled_goal = 4
models = dict(
    sac = SAC('MultiInputPolicy', make_env("sac"), verbose=1, target_update_interval=25),
    sac_her = SAC(
        'MultiInputPolicy',
        make_env("sac_her"),
        verbose=1, target_update_interval=25,
        replay_buffer_class=HerReplayBuffer,
        replay_buffer_kwargs=dict(
          n_sampled_goal=n_sampled_goal,
          goal_selection_strategy="future",
          # IMPORTANT: because the env is not wrapped with a TimeLimit wrapper
          # we have to manually specify the max number of steps per episode
          max_episode_length=25,
          online_sampling=True,
        ),
        buffer_size=int(1e6)
    ),
    ppo = PPO('MultiInputPolicy', make_env("ppo"), verbose=1, n_steps=25, n_epochs=80),
    a2c = A2C('MultiInputPolicy', make_env("a2c"), verbose=1, n_steps=25)
)

In [16]:
import torch

In [17]:
torch.cuda.is_available()

True

In [18]:
models_discrete = dict(
    sac = SAC('MultiInputPolicy', make_env_discrete("sac"), verbose=1, target_update_interval=25),
    sac_her = SAC(
        'MultiInputPolicy',
        make_env_discrete("sac_her"),
        verbose=1, target_update_interval=25,
        replay_buffer_class=HerReplayBuffer,
        replay_buffer_kwargs=dict(
          n_sampled_goal=n_sampled_goal,
          goal_selection_strategy="future",
          # IMPORTANT: because the env is not wrapped with a TimeLimit wrapper
          # we have to manually specify the max number of steps per episode
          max_episode_length=25,
          online_sampling=True,
        ),
        buffer_size=int(1e6)
    ),
    ppo = PPO('MultiInputPolicy', make_env_discrete("ppo"), verbose=1, n_steps=25, n_epochs=80),
    a2c = A2C('MultiInputPolicy', make_env_discrete("a2c"), verbose=1, n_steps=25)
)

Using cuda device
Wrapping the env in a DummyVecEnv.
Using cuda device
Wrapping the env in a DummyVecEnv.
Using cuda device
Wrapping the env in a DummyVecEnv.
Using cuda device
Wrapping the env in a DummyVecEnv.


In [19]:
# Separate env for evaluation
eval_env = CodenamesEnvHack(glove, wordlist)
eval_monitor = Monitor(eval_env)

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_monitor, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

NameError: name 'model' is not defined

In [19]:
# Separate env for evaluation
eval_env_discrete = CodenamesEnvHackDiscrete(glove, wordlist)
eval_monitor_discrete = Monitor(eval_env_discrete)
model_discrete = SAC('MultiInputPolicy', eval_monitor_discrete, verbose=1)

# Random Agent, before training
mean_reward_dc, std_reward_dc = evaluate_policy(model_discrete, eval_monitor_discrete, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward_dc:.2f} +/- {std_reward_dc}")

Using cuda device
Wrapping the env in a DummyVecEnv.
mean_reward=-32.40 +/- 5.4258639865002145


In [20]:
eval_monitor.episode_lengths

[]

In [21]:
np.mean([15, 16, 7, 4, 10, 2, 6, 5, 1, 8])

7.4

In [22]:
eval_monitor.get_episode_rewards()

[]

In [23]:
eval_monitor.episode_returns

[]

In [24]:
np.max(eval_monitor.episode_times)/len(eval_monitor.episode_times)

ValueError: zero-size array to reduction operation maximum which has no identity

In [20]:
from tqdm.auto import tqdm

class ProgressBarCallback(BaseCallback):
    """
    :param pbar: (tqdm.pbar) Progress bar object
    """
    def __init__(self, pbar):
        super(ProgressBarCallback, self).__init__()
        self._pbar = pbar

    def _on_step(self):
        # Update the progress bar:
        self._pbar.n = self.num_timesteps
        self._pbar.update(0)

# this callback uses the 'with' block, allowing for correct initialisation and destruction
class ProgressBarManager:
    def __init__(self, total_timesteps): # init object with total timesteps
        self.pbar = None
        self.total_timesteps = total_timesteps
        
    def __enter__(self): # create the progress bar and callback, return the callback
        self.pbar = tqdm(total=self.total_timesteps)
            
        return ProgressBarCallback(self.pbar)

    def __exit__(self, exc_type, exc_val, exc_tb): # close the callback
        self.pbar.n = self.total_timesteps
        self.pbar.update(0)
        self.pbar.close()

In [21]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq:
    :param log_dir: Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: Verbosity level.
    """
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          try:
              x, y = ts2xy(load_results(self.log_dir), 'timesteps')
              if len(x) > 0:
                  # Mean training reward over the last 100 episodes
                  mean_reward = np.mean(y[-100:])
                  if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

                  # New best model, you could save the agent here
                  if mean_reward > self.best_mean_reward:
                      self.best_mean_reward = mean_reward
                      # Example for saving best model
                      if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}")
                      self.model.save(self.save_path)
          except Exception:
            pass
        return True

In [22]:
class Run:
    def __init__(self, name, model, total_timesteps=1, mode="self_sim", num_eval_episodes=10, should_render=False, log_interval=500):
        assert mode in {"self_sim", "discrete"}
        print("Model:", name)
        self.log_dir = get_path_for(name, mode == "discrete")
        self.save_callback = SaveOnBestTrainingRewardCallback(check_freq=log_freq, log_dir=self.log_dir)
        self.logger = configure(get_path_for(name + "_logger", mode == "discrete"), ["stdout", "csv", "tensorboard"])
        print("Start training")
        model.set_logger(self.logger)
        with ProgressBarManager(total_timesteps) as progress_callback: # this the garanties that the tqdm progress bar closes correctly
            model.learn(total_timesteps=total_timesteps, log_interval=log_interval, callback=CallbackList([progress_callback, self.save_callback]))
        # Save the agent
        print("Saving...")
        model.save(f"{name}_codenames_hack_v0.0.1_end")
        # Evaluate the trained agent
        print("Evaluating...")
        if mode == "self_sim":
            self.eval_monitor_trained = make_env(name + "_eval")
        else:
            self.eval_monitor_trained = make_env_discrete(name + "_eval")
        mean_reward, std_reward = evaluate_policy(model, self.eval_monitor_trained, n_eval_episodes=num_eval_episodes, deterministic=True, render=should_render)
        print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")
        self.print_monitor_stat("Lengths", self.eval_monitor_trained.episode_lengths)
        self.print_monitor_stat("Returns", self.eval_monitor_trained.episode_returns)

    def print_monitor_stat(self, stat_name, stat):
        print(stat_name, stat, np.mean(stat), np.std(stat))

In [ ]:
runs = {k: Run(k, m, num_train_steps, should_render=True) for k, m in models.items()}

Model: sac
Logging to /mnt/ubuntu_data_hdd/school/masters/uwaterloo/1b/cs885/project/codenames/sb3_log/sac_logger
Start training


  0%|          | 0/1000000 [00:00<?, ?it/s]

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.62     |
|    ep_rew_mean     | -6.87    |
| time/              |          |
|    episodes        | 500      |
|    fps             | 58       |
|    time_elapsed    | 14       |
|    total_timesteps | 829      |
| train/             |          |
|    actor_loss      | 1.14     |
|    critic_loss     | 33.9     |
|    ent_coef        | 0.818    |
|    ent_coef_loss   | -2.95    |
|    learning_rate   | 0.0003   |
|    n_updates       | 728      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.8      |
|    ep_rew_mean     | -3.3     |
| time/              |          |
|    episodes        | 1000     |
|    fps             | 58       |
|    time_elapsed    | 29       |
|    total_timesteps | 1742     |
| train/             |          |
|    actor_loss      | 0.0762   |
|    critic_loss     | 16.8     |
|    ent_coef 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
runs_discrete = {k: Run(k, m, num_train_steps, mode="discrete", should_render=True) for k, m in models_discrete.items()}

In [ ]:
print

In [108]:
# Train the agent
with ProgressBarManager(10) as pmodels_discretecallback: # this the garanties that the tqdm progress bar closes correctly
    model_ppo.learn(total_timesteps=1, callback=CallbackList([progress_callback]))
# Save the agent
model_sac.save("sac_codenames_hack_v0.0.1")

  0%|          | 0/10 [00:00<?, ?it/s]

--------------------------
| time/              |   |
|    fps             | 1 |
|    iterations      | 1 |
|    time_elapsed    | 1 |
|    total_timesteps | 2 |
--------------------------


In [59]:
# Evaluate the trained agent
eval_monitor_trained = Monitor(eval_env)
mean_reward, std_reward = evaluate_policy(model, eval_monitor_trained, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/home/sherman/miniconda3/envs/fn_env/lib/python3.6/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward=-32.00 +/- 3.9749213828703582


In [158]:
print(eval_monitor.episode_lengths)

[15, 16, 7, 4, 10, 2, 6, 5, 1, 8]


In [159]:
print(eval_monitor.episode_returns)

[-39, -40, -31, -28, -34, -26, -30, -29, -25, -32]


In [160]:
print(eval_monitor.episode_times)

[9.813233137130737, 20.39705991744995, 25.02803874015808, 27.637056350708008, 34.15482449531555, 35.45982313156128, 39.39241051673889, 42.63814973831177, 43.291807651519775, 48.50733518600464]


In [ ]:
https://colab.research.google.com/github/Stable-Baselines-Team/rl-colab-notebooks/blob/master/monitor_training.ipynb#scrollTo=iaOPfOrwWEP4

In [ ]:
class PlotHack:
    def visualise_overall_agent_results(self, agent_results, agent_name, show_mean_and_std_range=False, show_each_run=False,
                                        color=None, ax=None, title=None, y_limits=None):
        """Visualises the results for one agent"""
        assert isinstance(agent_results, list), "agent_results must be a list of lists, 1 set of results per list"
        assert isinstance(agent_results[0], list), "agent_results must be a list of lists, 1 set of results per list"
        assert bool(show_mean_and_std_range) ^ bool(show_each_run), "either show_mean_and_std_range or show_each_run must be true"
        if not ax: ax = plt.gca()
        if not color: color =  self.agent_to_color_group[agent_name]
        if show_mean_and_std_range:
            mean_minus_x_std, mean_results, mean_plus_x_std = self.get_mean_and_standard_deviation_difference_results(agent_results)
            x_vals = list(range(len(mean_results)))
            ax.plot(x_vals, mean_results, label=agent_name, color=color)
            ax.plot(x_vals, mean_plus_x_std, color=color, alpha=0.1)
            ax.plot(x_vals, mean_minus_x_std, color=color, alpha=0.1)
            ax.fill_between(x_vals, y1=mean_minus_x_std, y2=mean_plus_x_std, alpha=0.1, color=color)
        else:
            for ix, result in enumerate(agent_results):
                x_vals = list(range(len(agent_results[0])))
                plt.plot(x_vals, result, label=agent_name + "_{}".format(ix+1), color=color)
                color = self.get_next_color()

        ax.set_facecolor('xkcd:white')

        # Shrink current axis's height by 10% on the bottom
        box = ax.get_position()
        ax.set_position([box.x0, box.y0 + box.height * 0.05,
                         box.width, box.height * 0.95])

        # Put a legend below current axis
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
                  fancybox=True, shadow=True, ncol=3)

        if not title: title = self.environment_name

        ax.set_title(title, fontsize=15, fontweight='bold')
        ax.set_ylabel('Rolling Episode Scores')
        ax.set_xlabel('Episode Number')
        self.hide_spines(ax, ['right', 'top'])
        ax.set_xlim([0, x_vals[-1]])

        if y_limits is None: y_min, y_max = self.get_y_limits(agent_results)
        else: y_min, y_max = y_limits

        ax.set_ylim([y_min, y_max])

        if self.config.show_solution_score:
            self.draw_horizontal_line_with_label(ax, y_value=self.config.environment.get_score_to_win(), x_min=0,
                                        x_max=self.config.num_episodes_to_run * 1.02, label="Target \n score")

In [ ]:
from stable_baselines3.common.monitor import load_results

In [ ]:
from stable_baselines3.common import results_plotter

In [ ]:
# Helper from the library
results_plotter.plot_results(
    [get_path_for("sac"), get_path_for("sac_her"), get_path_for("ppo"), get_path_for("a2c")],
    1e5, results_plotter.X_TIMESTEPS, "Similarity Matrix"
)
plt.legend(loc="best", labels=["SAC", "SAC (HER)", "PPO", "A2C"])

In [ ]:
# Helper from the library
tt = results_plotter.plot_results(
    [get_path_for("sac", True), get_path_for("sac_her", True), get_path_for("ppo", True), get_path_for("a2c", True)],
    1e5, results_plotter.X_TIMESTEPS, "Discrete"
)
plt.legend(loc="best", labels=["SAC", "SAC (HER)", "PPO", "A2C"])